Copyright ENEOS Corporation as contributors to Matlantis contrib project.

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import yaml

from ase import Atoms, units
from ase.build import bulk
from ase.optimize import BFGS
from ase.constraints import ExpCellFilter

In [2]:
from pfcc_extras.visualize.view import view_ngl
from pfcc_extras.structure.ase_rdkit_converter import smiles_to_atoms
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

/home/jovyan/.local/lib/python3.7/site-packages/pfp_api_client/__init__.py:36: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  f"New version of {package_name} is available. Please consider"


In [5]:
MODEL_VERSION='v3.0.0'
CALC_MODE=EstimatorCalcMode.CRYSTAL_PLUS_D3

def get_calculator(model_version=MODEL_VERSION, calc_mode=CALC_MODE):
    estimator = Estimator(model_version=model_version, calc_mode=calc_mode)
    calculator = ASECalculator(estimator)
    return calculator

In [3]:
# --- System settings
molecule_name = 'DMSO'
nmols = 50
name_smiles = {'DMSO':'CS(=O)C'}
bulk_name = 'Si'

repeat_mol = [3,3,3]
repeat_solid = [3,3,3]

# --- Initial geometry settings
fmax_opt = 0.03
steps_opt = 200
fmax_isolated = 0.01
fmax_liquid = 0.1
mol_cell = [5.0, 5.0, 5.0]

# --- MD settings ----
# Please change based on configuration
target_temp = 300
target_pressure = 101325 * units.Pascal
#steps_equilib = 1000000
#steps_product = 1000000
steps_equilib = 10000
steps_product = 10000
timestep_solid = 2.0
timestep_mol = 1.0
temperature = 300
log_interval = 10
traj_interval = 1
# --- Parameters
mass_density_factor = units._amu * 1e30 
huge = 999.0
# ------------------

In [4]:
inputdir = Path('./input')
os.makedirs(str(inputdir), exist_ok=True)

In [6]:
solid = bulk(bulk_name, cubic=True)

mol = smiles_to_atoms(name_smiles[molecule_name])
mol.cell = mol_cell
mol.pbc = True
mol.center()

mass_molecule = mol.get_masses().sum()
print(f'Number of atoms in the solid unit cell ({solid}): {len(solid)}')
print(f'Number of atoms per molecule ({mol}): {len(mol)}')

Number of atoms in the solid unit cell (Atoms(symbols='Si8', pbc=True, cell=[5.43, 5.43, 5.43])): 8
Number of atoms per molecule (Atoms(symbols='CSOCH6', pbc=True, cell=[5.0, 5.0, 5.0])): 10


In [7]:
solid.calc = get_calculator()
ecf = ExpCellFilter(solid)
opt = BFGS(ecf)
opt.run(fmax=fmax_opt, steps=steps_opt)
solid.cell = np.diag(solid.cell)

mol.calc = get_calculator()
opt = BFGS(mol)
opt.run(fmax=fmax_opt, steps=steps_opt)

      Step     Time          Energy         fmax
BFGS:    0 04:07:54      -38.938724        0.4838
BFGS:    1 04:07:54      -38.928702        1.4746
BFGS:    2 04:07:54      -38.940001        0.0132
      Step     Time          Energy         fmax
BFGS:    0 04:07:54      -33.408620        4.3725
BFGS:    1 04:07:54      -34.298702        5.0293
BFGS:    2 04:07:54      -35.776867        5.2553
BFGS:    3 04:07:55      -36.432342        3.7929
BFGS:    4 04:07:55      -36.840864        2.0869
BFGS:    5 04:07:55      -37.023653        2.4049
BFGS:    6 04:07:55      -37.141657        1.4355
BFGS:    7 04:07:55      -37.506547        1.4637
BFGS:    8 04:07:55      -37.636930        1.6686
BFGS:    9 04:07:55      -37.737400        1.1619
BFGS:   10 04:07:55      -37.839687        0.8001
BFGS:   11 04:07:55      -37.936608        0.9405
BFGS:   12 04:07:56      -38.016223        0.8348
BFGS:   13 04:07:56      -38.057030        0.5160
BFGS:   14 04:07:56      -38.082287        0.4186
BF

True

In [8]:
v = view_ngl([mol, solid], ['ball+stick'], h=300,replace_structure=True)
display(v)

In [9]:
solid.write(str(inputdir/"solid_initial.traj"))
mol.write(str(inputdir/"mol_initial.traj"))